In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
import pickle

from transformode import DataFrameOneHotEncoder 

%matplotlib inline

Import the dataset

In [2]:
df_orig = pd.read_csv('Raw Data/Loan_Default.csv')

In [3]:
loan_df = pd.read_csv('Raw Data/Loan_Default.csv')

In [3]:
#Get details, but also check data types for processing
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         148670 non-null  int64  
 1   year                       148670 non-null  int64  
 2   loan_limit                 145326 non-null  object 
 3   Gender                     148670 non-null  object 
 4   approv_in_adv              147762 non-null  object 
 5   loan_type                  148670 non-null  object 
 6   loan_purpose               148536 non-null  object 
 7   Credit_Worthiness          148670 non-null  object 
 8   open_credit                148670 non-null  object 
 9   business_or_commercial     148670 non-null  object 
 10  loan_amount                148670 non-null  int64  
 11  rate_of_interest           112231 non-null  float64
 12  Interest_rate_spread       112031 non-null  float64
 13  Upfront_charges            10

In [6]:
loan_df.groupby('Status')['Status'].describe()

count  mean  std  min  25%  50%  75%  max
Status                                              
0       112031.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1        36639.0   1.0  0.0  1.0  1.0  1.0  1.0  1.0

In [4]:
loan_df.groupby('Status')['rate_of_interest'].describe()

count      mean       std    min    25%     50%    75%  max
Status                                                                
0       112031.0  4.044931  0.561356  0.000  3.625  3.9900  4.375  8.0
1          200.0  4.350500  0.495546  3.125  3.990  4.3125  4.750  5.5

In [5]:
loan_df.groupby('Status')['Interest_rate_spread'].describe()

count      mean       std    min    25%     50%     75%    max
Status                                                                   
0       112031.0  0.441656  0.513043 -3.638  0.076  0.3904  0.7754  3.357
1            0.0       NaN       NaN    NaN    NaN     NaN     NaN    NaN

In [7]:
loan_df.groupby('Status')['Upfront_charges'].describe()

count         mean          std  min     25%      50%       75%  \
Status                                                                       
0       108875.0  3227.328554  3251.673989  0.0  584.39  2600.00  4815.315   
1          153.0  1565.237974  2299.820513  0.0    0.00   687.85  2310.750   

            max  
Status           
0       60000.0  
1       15584.0

In [9]:
loan_df.groupby('Status')['dtir1'].describe()

count       mean        std  min   25%   50%   75%   max
Status                                                             
0       104220.0  37.369267  10.027197  5.0  31.0  38.0  44.0  61.0
1        20329.0  39.597324  12.716828  5.0  32.0  42.0  49.0  61.0

In [10]:
loan_df.groupby('Status')['property_value'].describe()

count           mean            std     min       25%       50%  \
Status                                                                       
0       112029.0  505606.066286  342784.462653  8000.0  288000.0  428000.0   
1        21543.0  457786.009377  436255.032008  8000.0  228000.0  348000.0   

             75%         max  
Status                        
0       638000.0   9268000.0  
1       558000.0  16508000.0

In [11]:
loan_df.groupby('Status')['LTV'].describe()

count       mean        std       min        25%        50%  \
Status                                                                   
0       112029.0  72.064757  41.768899  2.072943  59.970817  74.504405   
1        21543.0  76.291468  28.580217  0.967478  64.273172  79.362416   

              75%      max  
Status                      
0       85.201149  7831.25  
1       90.948276  2956.25

Rate of interest, interest rate spread and upfront charges are all missing a lot of status = 1, so I am going to drop them as they likely are causing data leakage in my model which is why I'm getting 100% accuracy.

In [4]:
y = loan_df['Status']
loan_df = loan_df.drop(columns=['Status', 'ID', 'rate_of_interest','Interest_rate_spread', 'Upfront_charges'])

In [5]:
loan_df['year'] = str(loan_df['year'])
cat = loan_df[make_column_selector(dtype_include= object)]
dfohe = DataFrameOneHotEncoder()
catx = dfohe.fit_transform(cat)

In [6]:
numdf = loan_df[make_column_selector(dtype_include= np.number)]
ss = StandardScaler()
numx = pd.DataFrame(ss.fit_transform(numdf), columns=numdf.columns)

transformer = make_column_transformer(
                (StandardScaler(), make_column_selector(dtype_include = np.number)),
                (DataFrameOneHotEncoder(), make_column_selector(dtype_include = object)))
X = transformer.fit_transform(loan_df)


In [7]:
X = numx.join(catx)

In [16]:
numx.isna().sum()

loan_amount           0
term                 41
property_value    15098
income             9150
Credit_Score          0
LTV               15098
dtir1             24121
dtype: int64

In [8]:
X = X.fillna(value =0)

In [9]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2,  random_state=12)

In [11]:
test_X.to_csv('./testX.csv')
test_y.to_csv('./testy.csv')

Logistic Regression

In [56]:

#the grid of parameters to search over
parameters = {
    'penalty' : ['l1','l2'], 
    'C'       : np.logspace(-3,3,7),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
}


logreg = LogisticRegression(max_iter=1000)
clf = GridSearchCV(logreg,                    # model
                   param_grid = parameters,   # hyperparameters
                   scoring='accuracy',        # metric for scoring
                   cv=10) 


clf.fit(train_X,train_y)

print("Tuned Hyperparameters :", clf.best_params_)
print("Accuracy :",clf.best_score_)


/Users/kaylawilding/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
140 fits failed out of a total of 420.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/kaylawilding/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kaylawilding/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/kaylawilding/opt/anac

Tuned Hyperparameters : {'C': 100.0, 'penalty': 'l1', 'solver': 'liblinear'}
Accuracy : 0.8647592616191193


In [12]:
best_logreg = LogisticRegression(max_iter=1000, C=100, penalty='l1', solver = 'liblinear')
best_logreg.fit(train_X, train_y)
y_pred = best_logreg.predict(test_X)

print("Accuracy:",best_logreg.score(test_X, test_y))

Accuracy: 0.8635568709221766


In [30]:
decision_tree = DecisionTreeClassifier()

params = {
    "criterion" : ['gini', 'entropy'],
    "max_depth" : range(1,5),
    "min_samples_split" : range(2,5),
    "min_samples_leaf": range(1,5)
}
grid = GridSearchCV(decision_tree, 
                param_grid = params, 
                cv = 5,
                verbose = 1,
                n_jobs = -1)

grid.fit(train_X, train_y)
            

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 5),
                         'min_samples_leaf': range(1, 5),
                         'min_samples_split': range(2, 5)},
             verbose=1)

In [31]:
grid.best_estimator_

DecisionTreeClassifier(max_depth=4)

In [32]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 4,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [11]:
best_tree_class = DecisionTreeClassifier(criterion = 'gini', max_depth = 4, min_samples_split =2, min_samples_leaf = 1)
#best_tree_class = grid.best_estimator_
best_tree_class.fit(train_X, train_y)

y_pred_tree = best_tree_class.predict(test_X)

print("Accuracy:",best_tree_class.score(test_X, test_y))

Accuracy: 0.8754288020447972


In [36]:
best_tree_class.decision_path

<bound method BaseDecisionTree.decision_path of DecisionTreeClassifier(max_depth=4)>

In [35]:
importances = best_tree_class.feature_importances_
important_features = pd.DataFrame(importances[importances > 0],columns=['importance_val'])
important_feature_names = pd.DataFrame(X.columns[importances > 0], columns = ['feature_name'])
important_features_selected = pd.merge(important_features, important_feature_names, how = 'left', left_index = True, right_index = True)
important_features_selected

importance_val                                       feature_name
0        0.035333                                                LTV
1        0.062673                                              dtir1
2        0.040630                         loan_type[loan_type_type1]
3        0.003991       Neg_ammortization[Neg_ammortization_not_neg]
4        0.038157        lump_sum_payment[lump_sum_payment_not_lpsm]
5        0.816919                      credit_type[credit_type_EQUI]
6        0.002297  submission_of_application[submission_of_applic...

In [50]:
plt.figure(figsize = (20,20))
plot_tree(best_tree_class, feature_names = best_tree_class.feature_names_in_, fontsize=10)
plt.show()

In [55]:
groups = df_orig.groupby(['Status','credit_type'])
groups.size()

Status  credit_type
0       CIB            40542
        CRIF           36774
        EQUI               1
        EXP            34714
1       CIB             7610
        CRIF            7127
        EQUI           15297
        EXP             6605
dtype: int64

XGBoost Forest Model


In [37]:
train_X.columns = train_X.columns.str.replace("[[]", "_")
train_X.columns = train_X.columns.str.replace("[]]", "_")
train_X.columns = train_X.columns.str.replace("[<]", "less_than")
train_X.columns = train_X.columns.str.replace("[>]", "greater_than")
test_X.columns = test_X.columns.str.replace("[[]", "_")
test_X.columns = test_X.columns.str.replace("[]]", "_")
test_X.columns = test_X.columns.str.replace("[<]", "less_than")
test_X.columns = test_X.columns.str.replace("[>]", "greater_than")

/var/folders/cd/yhbft8yj611dsg2rkjqhhsv00000gn/T/ipykernel_64026/1188698962.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_X.columns = train_X.columns.str.replace("[[]", "_")
/var/folders/cd/yhbft8yj611dsg2rkjqhhsv00000gn/T/ipykernel_64026/1188698962.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_X.columns = train_X.columns.str.replace("[]]", "_")
/var/folders/cd/yhbft8yj611dsg2rkjqhhsv00000gn/T/ipykernel_64026/1188698962.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train_X.columns = train_X.columns.str.replace("[<]", "less_than")
/var/folders/cd/yhbft8yj611dsg2rkjqhhsv00000gn/T/ipykernel_64026/1188698962.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  train_X.columns = train_X.columns.str.replace("[>]", "greater_than")


Index(['loan_amount', 'term', 'property_value', 'income', 'Credit_Score',
       'LTV', 'dtir1',
       'year_year_0         2019\n1         2019\n2         2019\n3         2019\n4         2019\n          ... \n148665    2019\n148666    2019\n148667    2019\n148668    2019\n148669    2019\nName: year, Length: 148670, dtype: int64_',
       'loan_limit_loan_limit_cf_', 'loan_limit_loan_limit_ncf_',
       'loan_limit_loan_limit_nan_', 'Gender_Gender_Female_',
       'Gender_Gender_Joint_', 'Gender_Gender_Male_',
       'Gender_Gender_Sex Not Available_',
       'approv_in_adv_approv_in_adv_nopre_',
       'approv_in_adv_approv_in_adv_pre_', 'approv_in_adv_approv_in_adv_nan_',
       'loan_type_loan_type_type1_', 'loan_type_loan_type_type2_',
       'loan_type_loan_type_type3_', 'loan_purpose_loan_purpose_p1_',
       'loan_purpose_loan_purpose_p2_', 'loan_purpose_loan_purpose_p3_',
       'loan_purpose_loan_purpose_p4_', 'loan_purpose_loan_purpose_nan_',
       'Credit_Worthiness_Credit

In [43]:
# fit model on training data
parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}
xgbmodel = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)
grid_search = GridSearchCV(
    estimator=xgbmodel,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 10,
    verbose=True
)
grid_search.fit(train_X, train_y)

Fitting 10 folds for each of 96 candidates, totalling 960 fits


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None, nthread=4,
                                     num_parallel_tree=None, predictor=None, ...),
             n_jobs=10,
             param_grid={'learning_rate': [0.1, 0.01, 0.05],
                         'max_depth': range(2, 10),
                         'n_estimators': range(60, 220, 40)},
             scoring='roc_auc', verbose=True)

In [44]:
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=7, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=180, n_jobs=4, nthread=4,
              num_parallel_tree=1, predictor='auto', ...)

In [46]:
# Dictionary of best parameters
best_xgb_params = grid_search.best_params_
# Best XGB model that was found based on the metric score you specify
best_xgb_model = grid_search.best_estimator_
# Save model
pickle.dump(grid_search.best_estimator_, open("xgb_model.pickle", "wb"))

In [47]:
# make predictions for test data
y_pred = best_xgb_model.predict(test_X)
predictions = [round(value) for value in y_pred]

In [48]:
# evaluate predictions
accuracy = accuracy_score(test_y, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 89.71%
